In [1]:
import carla    # CARLA Simulator
import random   # Randomness to pick spawn points
import cv2      # OpenCV for image processing to work with images from cameras
import numpy as np # Numpy to work with data - re-shaping images representations

In [2]:
# connect to the simulator
client = carla.Client('localhost', 2000)

In [3]:
# define enviroment/world and get possible places to pawn a car
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

In [4]:
# look for a blueprint of Mini Car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

In [5]:
# sapwn a car in a random location
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point) # spawn a car in the world and get a reference to it (actor) 

In [7]:
# move the simulator view to the car
spectator = world.get_spectator()
start_point.location.z = start_point.location.z + 1  # move the camera a bit higher than the car
spectator.set_transform(start_point) # move the camera to the car

In [8]:
print(vehicle.type())

AttributeError: 'NoneType' object has no attribute 'type'

In [14]:
# send the car off the auto pilot - this will leave the spectator
vehicle.set_autopilot(True)

AttributeError: 'NoneType' object has no attribute 'set_autopilot'

In [ ]:
# Setting RGB camera

CAMERA_POS_Z = 1.6 #this means 1.6m up from the ground
CAMERA_POS_X = 0.9 #this is 0.9m forward

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

In [ ]:
# Setting Segmentation camera

while True:
    # Display with imshow
    cv2.imshow('All cameras', camera_delta['image'])

    # break loop if user presses 'q'
    if cv2.waitkey(1) == ord('q'):
        break   

cv2.destroyAllWindows()

In [ ]:
# Grab a snapshot from a camera and show it in a pop-up window
img = camera_data['image'] # get the image from the dictionary
cv2.imshow('RGB Camera', img) # show the image in a pop-up window
cv2.waitKey(0) # wait for a key press before closing the window

In [ ]:
# clean up after yourself

camera.stop() # stop the camera - this is the opposite of camera.listen()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy() # destroy all vehicles
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy() # destroy all sensors

In [ ]:
start_point = spawn_points[0] # get the first spawn point
spectator = world.get_spectator() # get the spectator
spectator.set_transform(start_point) # move the camera to the car

In [ ]:
spectator.set_transform(carla.Transform(carla.Location(x=-1085.286377, y=3112.225830, z=356.060608), carla.Rotation(pitch=1.648070, yaw=20.234367, roll=0.000000))) # move the camera to the car - this is a specific location in the map

In [ ]:
print(start_point)